In [45]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import numpy as np

import torch.utils.data as torchdata
import torchvision.transforms as transforms
import torchvision.datasets as vision_datasets

import nltk
from torchtext.data import Field, Iterator
import torchtext.datasets as text_datasets

torch.manual_seed(1)

import matplotlib.pyplot as plt
%matplotlib inline

### Reference 

* https://github.com/yunjey/pytorch-tutorial/blob/master/tutorials/01-basics/pytorch_basics/main.py
* https://github.com/pytorch/vision
* https://github.com/pytorch/text
* http://anie.me/On-Torchtext/
* http://torchtext.readthedocs.io/en/latest/

##  torch.utils.data.Dataset

데이터셋을 핸들링하기 위한 (추상) 클래스

In [157]:
class CustomDataset(torchdata.Dataset):
    def __init__(self):
        
        # 데이터 로드, 전처리
        # or 데이터 경로
        self.x = torch.randn(100,1).float()
        self.y = torch.arange(0,100).long()
        
    def __getitem__(self, index):
        # 인덱스에 해당하는 데이터셋 리턴
        return self.x[index], self.y[index]
        
    def __len__(self):
        # 데이터셋 수
        return len(self.x) 

In [158]:
mydataset = CustomDataset()

### torch.utils.data.DataLoader 

데이터 이터레이터

In [161]:
train_loader = torchdata.DataLoader(dataset=mydataset,
                                           batch_size=5, 
                                           shuffle=True)

In [162]:
for i,batch in enumerate(train_loader):
    print(batch)
    break

[
-0.4585
 0.7840
-0.0157
-0.0704
 0.8635
[torch.FloatTensor of size 5x1]
, 
 69
 32
 76
 81
 37
[torch.LongTensor of size 5]
]


## torchvision 

In [38]:
train_dataset = vision_datasets.MNIST(root='../../data/MNIST/',
                               train=True, 
                               transform=transforms.ToTensor(),
                               download=True)

# Select one data pair (read data from disk).
image, label = train_dataset[0]
print (image.size())
print (label)

# Data Loader (this provides queue and thread in a very simple way).
train_loader = torchdata.DataLoader(dataset=train_dataset,
                                           batch_size=100, 
                                           shuffle=True,
                                           num_workers=2)

torch.Size([1, 28, 28])
5


### transform

In [3]:
totensor = transforms.ToTensor() # PIL 이미지나 H,W,C의 numpy.ndarray ==> C,H,W FloatTesnor
toimage = transforms.ToPILImage() # C,H,W FloatTensor ==> PIL 이미지

In [8]:
image = toimage(train_dataset[0][0])
print(image.size)
image

(28, 28)


In [9]:
tensor = totensor(image)
tensor.size()

torch.Size([1, 28, 28])

### Build Iterator

In [12]:
for inputs,targets in train_loader:
    print(inputs.size()) # 배치사이즈, 채널수, 높이, 너비
    print(targets.size())
    break

torch.Size([100, 1, 28, 28])
torch.Size([100])


## torchtext 

### 토크나이저 

In [2]:
nltk.word_tokenize("Hi, my name is sungdong. What's your name?")

['Hi',
 ',',
 'my',
 'name',
 'is',
 'sungdong',
 '.',
 'What',
 "'s",
 'your',
 'name',
 '?']

### Field 선언 

In [3]:
TEXT = Field(tokenize=nltk.word_tokenize,lower=True, include_lengths=True, batch_first=True) # B,T
LABEL = Field(sequential=False) # 토크나이저 사용 x

In [23]:
train, test = text_datasets.IMDB.splits(TEXT, LABEL,root='../../data/IMDB/')

downloading aclImdb_v1.tar.gz


In [26]:
print(train.examples[0].text[:100])
print(train.examples[0].label)

['based', 'on', 'elmore', 'leonard', ',', 'this', 'is', 'a', 'violent', 'and', 'intelligent', 'action', 'film', '.', 'the', 'story', ':', 'a', 'business', 'man', 'is', 'blackmailed', 'by', 'some', '3', 'criminals', '.', 'roy', 'schieder', 'does', 'great', 'job', 'as', 'the', 'leading', 'character', 'and', 'special', 'credit', "'s", 'got', 'to', 'go', 'to', 'john', 'glover', 'who', 'plays', 'sort', 'of', 'a', 'naughty', 'psychopath', '.', 'i', 'must', 'mention', 'that', 'the', 'villains', 'characters', 'are', 'very', 'complex', 'and', 'interesting', '-', 'something', 'that', 'is', 'very', 'rare', 'for', 'an', 'action', 'film', '.', 'also', 'features', 'some', 'beautiful', 'and', 'sexy', 'women', '-', 'most', 'notable', 'are', 'kelly', 'preston', 'as', 'the', 'young', 'bate', 'for', 'schieder', "'s", 'character', '.', 'vanity']
pos


### Build vocab 

In [28]:
TEXT.build_vocab(train,min_freq=2) 
LABEL.build_vocab(train)

In [29]:
len(TEXT.vocab)

56574

In [37]:
print(TEXT.vocab.itos[:10]) # index to word
print(list(TEXT.vocab.stoi.items())[:10]) # word to index

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']
[('showgirls', 12279), ('stodgy', 22196), ('toothbrush', 31320), ('gert', 26877), ('unsafe', 31419), ('pinching', 40837), ('spy-thriller', 54415), ('ceremonies', 12797), ('johns', 16759), ('..well', 22406)]


###  Build Iterator 

In [65]:
# make iterator for splits
train_iter, test_iter = Iterator.splits(
    (train, test), batch_size=3, device=-1) # device -1 : cpu, device 0 : 남는 gpu

In [66]:
for batch in train_iter:
    print(batch.text[0].size()) # 배치 사이즈, 최대 길이에 패딩 맞춰진 단어 index (LongTensor)
    print(batch.text[1])
    print(batch.label)
    break

torch.Size([3, 396])

 396
 336
 339
[torch.LongTensor of size 3]

Variable containing:
 2
 1
 1
[torch.LongTensor of size 3]

